# Code Willing CDS Technical Interview

Below is historical raw security reference data. This data is a time series that contains every security that has traded on the New York Stock Exchange and the various identifiers that are associated with them. It also has a change date column, `Changed`, that lists the date of the changes to the identifiers over time. These identifiers are used to join different datasets or tables together when building a quantatative model. We will explore some of these identifiers below:

In [106]:
import numpy as np
import pandas as pd

pd.set_option('mode.chained_assignment', 'raise')
pd.options.display.max_columns = None
pd.options.mode.use_inf_as_na = True

In [107]:
sample = pd.read_csv('../data/pit_sample_xnys.txt', sep='\t', skiprows=1)
sample = sample[(sample.SecID.notna()) & (sample.IssID.notna())] # filter out rows with missing identifiers
sample.loc[:, ['SecID', 'IssID']] = sample.loc[:, ['SecID', 'IssID']].astype(int) # convert id columns to int
sample.head()

/var/folders/3w/vg5tdhqd3fx1m3n150wrlhhh0000gn/T/ipykernel_72846/3449895685.py:3: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  sample.loc[:, ['SecID', 'IssID']] = sample.loc[:, ['SecID', 'IssID']].astype(int) # convert id columns to int


,ActFlag,Changed,SecID,IssID,Isin,Uscode,FISN,IssuerName,CntryofIncorp,CntryIncorpNumber,CntryofDom,StateofDom,LEI,FinancialYearEnd,ShellComp,SIC,CIK,GICS,NAICS,CIC,CFIcode,SectyCD,SecurityDesc,StructCD,ParValue,PVCurrency,StatusFlag,PrimaryExchgCD,Voting,VotePerSec,SharesOutstanding,EffectiveDate
0,U,2022/08/03 14:14:40,1399,1379,PHY078321437,NaN,NaN,Benguet Corporation,PH,NaN,NaN,NaN,NaN,3112.0,NaN,NaN,11290.0,15104020.0,212311.0,PH31,ESVUFR,EQS,Ordinary Shares - Class B,NaN,1.00,PHP,A,PHPSE,V,1.0,247926121.0,2021/10/08
1,U,2022/08/23 04:41:39,2234,2112,CA8667961053,866796105,NaN,"Sun Life Financial, Inc.",CA,NaN,NaN,NaN,549300V040JZX5S66Z78,3112.0,NaN,6311.0,1097362.0,40301020.0,524113.0,CA31,ESVTFR,EQS,Ordinary Shares,NaN,NaN,CAD,A,CATSE,V,1.0,586079916.0,2022/08/22
2,U,2009/12/14 22:54:13,4962,4788,GB0003188660,NaN,NaN,Enterprise Oil plc,GB,NaN,NaN,NaN,213800WIHSFSGMYPTK39,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EQS,Ordinary Shares,NaN,0.25,GBP,I,GBLSE,V,NaN,NaN,NaN
3,U,2019/11/15 17:29:28,5246,5050,GB0004089834,NaN,NaN,Hanson plc,GB,NaN,NaN,NaN,529900N6RQR0PX7ZTR62,NaN,NaN,3822.0,NaN,NaN,334512.0,NaN,ESVUFR,EQS,Ordinary Shares,NaN,2.00,GBP,I,NaN,V,NaN,736968849.0,NaN
4,U,2010/08/20 18:43:01,5922,5692,GB0004437462,NaN,NaN,Powergen plc,GB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EQS,Ordinary Shares,NaN,0.50,GBP,I,GBLSE,V,NaN,NaN,NaN


In [3]:
sample.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10306 entries, 0 to 10305
Data columns (total 32 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ActFlag            10306 non-null  object 
 1   Changed            10306 non-null  object 
 2   SecID              10306 non-null  int64  
 3   IssID              10306 non-null  int64  
 4   Isin               9946 non-null   object 
 5   Uscode             9881 non-null   object 
 6   FISN               13 non-null     object 
 7   IssuerName         10306 non-null  object 
 8   CntryofIncorp      10306 non-null  object 
 9   CntryIncorpNumber  25 non-null     object 
 10  CntryofDom         1845 non-null   object 
 11  StateofDom         1665 non-null   object 
 12  LEI                5463 non-null   object 
 13  FinancialYearEnd   8508 non-null   float64
 14  ShellComp          23 non-null     object 
 15  SIC                7466 non-null   float64
 16  CIK                846

Below we made a list of the important identifers in the dataset, as well as the change date. `SecID` is the security identifier, `IssID` is the company identifier, `Isin` is an international identifier, and `SIC, CIK, GICS, and NAICS` are all additional identifiers used to join datasets together.

In [4]:
ids = ['IssuerName', 'Changed', 'SecID', 'IssID', 'CntryofIncorp', 'Isin', 'SIC', 'CIK', 'GICS', 'NAICS']

In [5]:
example = (sample.query('IssID == 2042')
                 .loc[:, ids]
                 .sort_values(['IssID', 'Changed'])
                 .reset_index(drop=True))
example

,IssuerName,Changed,SecID,IssID,CntryofIncorp,Isin,SIC,CIK,GICS,NAICS
0,PLDT Inc,2006/01/11 21:16:24,89317,2042,PH,US7182527033,4813.0,78150.0,50102010.0,517210.0
1,PLDT Inc,2022/07/11 05:06:21,59927,2042,PH,US69344D4088,4813.0,78150.0,50102010.0,517210.0


In [6]:
example2 = (sample.query('IssID == 4121')
                 .loc[:, ids]
                 .sort_values(['IssID', 'Changed'])
                 .reset_index(drop=True))
example2

,IssuerName,Changed,SecID,IssID,CntryofIncorp,Isin,SIC,CIK,GICS,NAICS
0,Santander UK Plc,2003/12/02 10:05:46,92117,4121,GB,US0029202050,6029.0,1087711.0,40101010.0,522110.0
1,Santander UK Plc,2006/11/28 22:50:22,75715,4121,GB,US0029207000,6029.0,1087711.0,40101010.0,522110.0
2,Santander UK Plc,2007/10/08 13:00:37,83287,4121,GB,US0029208099,6029.0,1087711.0,40101010.0,522110.0


One of the main issues we address is when a company undergoes a corporate action (such as a split or merger), the company's underlying securities will often change identifiers. However, if you owned the security before the corporate action, you will still own it afterwards. In the above examples, the `SecID` changes, when it likely should have remained the same.

You can see for both companies, the `SecID` and `ISIN` changed on different dates, but the rest of the identifiers remained the same. This likely means that these sets of rows for `IssID == 2042` and `IssID == 4121` are actually the same security, and can be joined by our `CWIQ code`. Below I will provide a quick example of what this looks like:

In [7]:
_example = example.assign(cwiq_code=1001)
_example2 = example2.assign(cwiq_code=1002)
sec_master = pd.concat([_example, _example2])
sec_master.head()

,IssuerName,Changed,SecID,IssID,CntryofIncorp,Isin,SIC,CIK,GICS,NAICS,cwiq_code
0,PLDT Inc,2006/01/11 21:16:24,89317,2042,PH,US7182527033,4813.0,78150.0,50102010.0,517210.0,1001
1,PLDT Inc,2022/07/11 05:06:21,59927,2042,PH,US69344D4088,4813.0,78150.0,50102010.0,517210.0,1001
0,Santander UK Plc,2003/12/02 10:05:46,92117,4121,GB,US0029202050,6029.0,1087711.0,40101010.0,522110.0,1002
1,Santander UK Plc,2006/11/28 22:50:22,75715,4121,GB,US0029207000,6029.0,1087711.0,40101010.0,522110.0,1002
2,Santander UK Plc,2007/10/08 13:00:37,83287,4121,GB,US0029208099,6029.0,1087711.0,40101010.0,522110.0,1002


In [8]:
sec_master.query('cwiq_code == 1001')

,IssuerName,Changed,SecID,IssID,CntryofIncorp,Isin,SIC,CIK,GICS,NAICS,cwiq_code
0,PLDT Inc,2006/01/11 21:16:24,89317,2042,PH,US7182527033,4813.0,78150.0,50102010.0,517210.0,1001
1,PLDT Inc,2022/07/11 05:06:21,59927,2042,PH,US69344D4088,4813.0,78150.0,50102010.0,517210.0,1001


In [9]:
sec_master.query('cwiq_code == 1002')

,IssuerName,Changed,SecID,IssID,CntryofIncorp,Isin,SIC,CIK,GICS,NAICS,cwiq_code
0,Santander UK Plc,2003/12/02 10:05:46,92117,4121,GB,US0029202050,6029.0,1087711.0,40101010.0,522110.0,1002
1,Santander UK Plc,2006/11/28 22:50:22,75715,4121,GB,US0029207000,6029.0,1087711.0,40101010.0,522110.0,1002
2,Santander UK Plc,2007/10/08 13:00:37,83287,4121,GB,US0029208099,6029.0,1087711.0,40101010.0,522110.0,1002


As you can see, our cwiq_code allows us to query the data and pull all relevant information for a security, even if the vendor considers them two (or more) separate securities.

Your task is to provide:
- A data quality analysis on the data to determine the best way to identify which securities should be joined together. In the examples above, all the identifiers outside of `SecID` and `Isin` remained the same. But in other cases, other identifiers might changes as well, such as `GICS` or `SIC`. If `IssID` is the same, but the rest of the identifiers are different, then the two `SecID`s likely should get their own `cwiq_code` and not be joined together. Determine which columns are the best for determining if the securities are the same. In the cells below, show your analysis in determining the best way to identify these securities.
- Once you complete your analysis, develop a function that will
    1. Read in the data
    2. Extract the securities that need to be joined by `cwiq_code`
    3. Output a csv file of these securities to the folder called `output/`.

There is not one correct answer for this analysis. We are mostly concerned about seeing your programming abilities in python, and well as how you approach problems.

*You do not need to provide the `cwiq_code` itself, just the analysis, function, and output file of the securities that need to be joined*

The output below is an example of what your data should look like when you output it to csv

In [10]:
pd.concat([example, example2])

,IssuerName,Changed,SecID,IssID,CntryofIncorp,Isin,SIC,CIK,GICS,NAICS
0,PLDT Inc,2006/01/11 21:16:24,89317,2042,PH,US7182527033,4813.0,78150.0,50102010.0,517210.0
1,PLDT Inc,2022/07/11 05:06:21,59927,2042,PH,US69344D4088,4813.0,78150.0,50102010.0,517210.0
0,Santander UK Plc,2003/12/02 10:05:46,92117,4121,GB,US0029202050,6029.0,1087711.0,40101010.0,522110.0
1,Santander UK Plc,2006/11/28 22:50:22,75715,4121,GB,US0029207000,6029.0,1087711.0,40101010.0,522110.0
2,Santander UK Plc,2007/10/08 13:00:37,83287,4121,GB,US0029208099,6029.0,1087711.0,40101010.0,522110.0


To submit this code, please follow these steps:
    
1. Create a new git repository called "cw-technical-{first_name}_{last_name}" in either gitlab or github (your choice)
2. Ensure the repository you created is public
3. Add your code and altered data to this repository. The code should be in a folder called `src/` and the data should be in a folder called `output/`. Push these changes.
4. Please forward the link to your public git repository to dalton.hall@codewilling.com and cody.bordelon@codewilling.com

# Data Analysis

### Introduction

In order to explore the data and variables, I have investigated the following as part of my data analysis:
- How many unique `IssID`s and `SecID`s can we observe in the dataset?
- Is there a pattern in the way other identifiers and `Changed` are linked with `IssID`?
- What are the datatypes for each column, and how many missing values do we have for the various identifiers
- For securities with the same `IssID`, do `Isin`s repeat?
- For securities with the same `IssID`, is `CntryofIncorp` always the same?
- What type of codes are `SIC`, `CIK`, `GICS` and `NAICS`?

As part of my analysis, and for the sake of brevity, I have made the following assumptions:
- I'm only looking at the variables you have mentioned as important.
- Values that are missing in identifier columns are missing at random
- If the `IssID` is the same for 2 securities but every other identifier is different, then I consider them to be different securities that would require different cwiq codes.
- 2 securities do not have the same `IssID` **and** the same `Isin` (true for this sample)
- If the `IssID` for 2 or more securities is the same, I check their `SIC`, `CIK`, `GICS` and `NAICS`. If all 4 of these columns are the same for these securities, then I consider that to be the same security, and assign them the same cwiq code.
- I drop the rows where `SIC`, `CIK`, `GICS` and `NAICS` are NA, but depending on context, this will obviously change.
  

In [132]:
## Creating a new dataframe that consists only of important variables
filtered_sample = sample[['IssuerName', 'Changed', 'SecID', 'IssID', 'CntryofIncorp', 'Isin', 'SIC', 'CIK', 'GICS', 'NAICS']]

print(f'There are {filtered_sample.shape[0]} raw securities in this sample and {filtered_sample.SecID.nunique()} security IDs.')

There are 10306 raw securities in this sample and 10306 security IDs.


In [133]:
filtered_sample.head()

,IssuerName,Changed,SecID,IssID,CntryofIncorp,Isin,SIC,CIK,GICS,NAICS
0,Benguet Corporation,2022/08/03 14:14:40,1399,1379,PH,PHY078321437,NaN,11290.0,15104020.0,212311.0
1,"Sun Life Financial, Inc.",2022/08/23 04:41:39,2234,2112,CA,CA8667961053,6311.0,1097362.0,40301020.0,524113.0
2,Enterprise Oil plc,2009/12/14 22:54:13,4962,4788,GB,GB0003188660,NaN,NaN,NaN,NaN
3,Hanson plc,2019/11/15 17:29:28,5246,5050,GB,GB0004089834,3822.0,NaN,NaN,334512.0
4,Powergen plc,2010/08/20 18:43:01,5922,5692,GB,GB0004437462,NaN,NaN,NaN,NaN


In [11]:
## Now let's see how many unique IssIDs we have
print(f'Of the {filtered_sample.shape[0]} raw securities, there are only {filtered_sample.IssID.nunique()} unique company identifiers, IssIDs')

Of the 10306 raw securities, there are only 6862 unique company identifiers, IssIDs


In [134]:
## Checking datatypes
filtered_sample.dtypes

IssuerName        object
Changed           object
SecID              int64
IssID              int64
CntryofIncorp     object
Isin              object
SIC              float64
CIK              float64
GICS             float64
NAICS            float64
dtype: object

For now, we can use these datatypes as they are.

The `IssuerName` and `IssID` are the same for all rows, a quick check reveals.  

In [40]:
## Here we check how many missing values in other columns are there?
filtered_sample[['SIC', 'CIK', 'GICS', 'NAICS', 'Isin']].isna().sum()

SIC      2840
CIK      1842
GICS     4285
NAICS    3285
Isin      360
dtype: int64

This shows we cannot always depend on these columns to give us reliable information, since sometimes they are missing.

We start with the largest level of granularity that is **complete** in the dataset, `IssID`. We know that securities with the same IssID *can* essentially be the same security, and so we want to focus our analysis on these rows.

We also observe that `Isin` is a security specific ID, whereas the others are industry classifications and are bound to repeat or change over time. If two rows have the repeating IDs for industry classification systems, we could assume it is referring to the same security.


For the next step, since there are duplicates in IssID, we need to find which ones are repeating. I create a new dataset of raw securities with repeating IssIDs.

In [85]:
## We first create a column that counts the number of times IssID repeats
grouped = filtered_sample.groupby('IssID').size().reset_index(name='counts')

## Then we get a unique list of these repeating IssIDs
duplicated_issids = grouped[grouped['counts'] > 1]['IssID']

# Now we filter to obtain the new dataset that has rows with IssIDs that appear more than once
duplicated_data = filtered_sample[filtered_sample['IssID'].isin(duplicated_issids)]

In [86]:
print(f'This gives us a dataset with {duplicated_data.shape[0]} rows.')

This gives us a dataset with 4834 rows.


We also check below if the `CntryofIncorp` is consistent for each `IssID` in this data.

In [87]:
# group data by IssID and count number of unique values in country column
grouped_c = duplicated_data.groupby('IssID')['CntryofIncorp'].nunique().reset_index(name='unique_countries')

# identify IssIDs with more than one unique country
different_countries = grouped_c[grouped_c['unique_countries'] > 1]['IssID']

if different_countries.empty:
    print("The corresponding value of country is the same for all rows with the same IssID.")
else:
    print("The corresponding value of country is different for at least one row with the same IssID.")


The corresponding value of country is the same for all rows with the same IssID.


In [88]:
# Before looking at other codes, I drop rows with NA values in SIC, CIK, NAICS and GICS
duplicated_data_dropped_na = duplicated_data.dropna(subset=['SIC', 'CIK','GICS', 'NAICS'])

The next step is to check that for rows with the same `IssID`, are the other identifier codes the same as well. We do this by creating a table grouped by `IssID` and other codes, and count the number of unique occurences. We then filter the original dataset to these repeating combinations and store them in a dataframe called `need_cwiq`, indicating that these rows should be joined by the same cwiq_code.

(Note: We can do this first with `IssID` and `Isin` unique combinations as well, but in this dataset, they don't exist. Hence, I've excluded them from this analysis.)

In [131]:
# Group the data by IssID and other identifiers and count the number of occurrences
counts = duplicated_data_dropped_na.groupby(['IssID', 'SIC', 'CIK','GICS', 'NAICS']).size().reset_index(name='counts_unique')

# Filter the rows where the count is greater than 1
repeats = counts[counts['counts_unique'] > 1]

# Merge the repeating rows with the original dataset to get the full rows
result = pd.merge(filtered_sample, repeats, how = 'left', on=['IssID', 'SIC', 'CIK','GICS', 'NAICS'])

## Filter out the rows that need a cwiq code
need_cwiq = result.loc[result.counts_unique > 1]

We save this dataframe as a csv, to the output folder.

# ETL Function

In [120]:
## Helper function to pre-processes data
def preprocess(dataframe):
    
    dataframe = dataframe[(dataframe.SecID.notna()) & (dataframe.IssID.notna())] # filter out rows with missing identifiers
    
    ## Filter to columns needed 
    filtered_df = dataframe[['IssuerName', 'Changed', 'SecID', 'IssID', 'CntryofIncorp', 'Isin', 'SIC', 'CIK', 'GICS', 'NAICS']]
    
    ## Add this print statement to check how many rows there are in the full data
    print(f'There are {filtered_df.shape[0]} raw securities in this sample and {filtered_df.SecID.nunique()} security IDs.')
    
    ## We first create a column that counts the number of times IssID repeats
    grouped_issid = filtered_df.groupby('IssID').size().reset_index(name='counts_issid')

    ## Then we get a unique list of these repeating IssIDs
    duplicated_issids = grouped_issid[grouped_issid['counts_issid'] > 1]['IssID']

    # Now we filter to obtain the new dataset that has rows with IssIDs that appear more than once
    duplicated_data = filtered_df[filtered_df['IssID'].isin(duplicated_issids)]

    return duplicated_data
     

In [126]:
## MAIN ETL Function
def etl_function(source_file, destination_file):
    
    # Extract
    data = pd.read_csv(source_file, sep='\t', skiprows=1)

    # Pre-process Data
    print('Pre-processing data....')
    pre_processed = preprocess(data)
    print('Pre-processing complete.')
    
    # drop rows with NA values in SIC, CIK, NAICS and GICS
    pre_processed_no_na = pre_processed.dropna(subset=['SIC', 'CIK','GICS', 'NAICS'])
    
    print('NAs for SIC, CIK, GICS, NAICS dropped.')
    
    ## FIND SAME SECURITIES
    # Group the data by IssID and other identifiers and count the number of occurrences
    counts = pre_processed_no_na.groupby(['IssID', 'SIC', 'CIK','GICS', 'NAICS']).size().reset_index(name='counts_unique')

    # Filter the rows where the count is greater than 1
    repeats = counts[counts['counts_unique'] > 1]

    # Merge the repeating rows with the original dataset to get the full rows
    result = pd.merge(pre_processed, repeats, how = 'left', on=['IssID', 'SIC', 'CIK','GICS', 'NAICS'])

    # Find rows which are the same securities
    need_cwiq_code = result.loc[result.counts_unique > 1].drop(columns='counts_unique')

    # Load
    print('Saving data to csv...')
    need_cwiq_code.to_csv(destination_file, index=False)

    print("ETL process complete.")


## Saving the data to output

In [127]:
etl_function('../data/pit_sample_xnys.txt', '../output/sample_output.csv')

Pre-processing data....
There are 10306 raw securities in this sample and 10306 security IDs.
Pre-processing complete.
NAs for SIC, CIK, GICS, NAICS dropped.
Saving data to csv...
ETL process complete.


## Checking the saved output

In [128]:
checking_output = pd.read_csv('../output/sample_output.csv')
checking_output.head()

,IssuerName,Changed,SecID,IssID,CntryofIncorp,Isin,SIC,CIK,GICS,NAICS
0,Tsakos Energy Navigation Limited,2022/08/08 04:38:32,18665.0,18313.0,BM,BMG9108L1735,4412.0,1166663.0,10102040.0,483111.0
1,Chubb Limited,2022/08/08 04:38:32,18679.0,18327.0,CH,CH0044328745,6331.0,896159.0,40301040.0,524126.0
2,Deutsche Bank AG,2022/03/28 04:51:45,20946.0,20481.0,DE,DE0005140008,6029.0,948046.0,40203030.0,522110.0
3,Aaron`s Inc,2020/10/19 14:09:38,32715.0,29299.0,US,US0025353006,7359.0,706688.0,25504060.0,442110.0
4,Aaron`s Inc,2010/12/13 12:42:04,32716.0,29299.0,US,US0025352016,7359.0,706688.0,25504060.0,442110.0
